Hey Hi,

Welcome to my first Competition Notebook as well as my first notebook published on Kaggle.com

As **Kaggle** is one of the most favorite Data Science practice and compete with other fellow Data Scientists. 

In 09/01/2021 to 10/04/2021 an survey was conducted on **Kaggle.com** by **Kaggle**. The survey was about **conduct an industry-wide survey** that present comprehensive view of the **state of data science and machine learning**.

The purpose of this note book is to **provide Story of Data Science and Machine Learning**, I tried my best to give insights on each and every **Question** through Visualization of every questions mmentioned in **Supplements** as well as **Dataset**.

Gothrough it and if you find this notebook useful please **UpVote** and you like my code please try yourself. 

And if their is something is missing please give your insights by commenting....


# Loading Required Packages

In [ ]:
import os
import re
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import poltly.io as pio
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected=True)
# pio.templates.default = "none"


# Loading dataset

In [ ]:
# Loading Original Data
kag_surv_2021 = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv')


# Making copy of original data
data = kag_surv_2021.copy()

# Data Exploration

**Checking Head and Tail Records**

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
df = data.copy().drop(0, axis = 0)

# Stripping Whitespaces
for col in df.columns:
    df[col] = df[col].str.strip()


**Converting Time from Minutes and Seconds**

**Reusable Functions**

In [ ]:

def clean(data,Question,n):
    """
    Get counts of Attributes from features and returns Dataframe with Counts
    """
    x = []
    for i in range(1,n):
        x.append(Question+str(i))
    x.append(str('Q7_OTHER'))

    ddict = {}
    for i in x:
        ddict[data[i].value_counts().keys()[0]]=data[i].value_counts()[0]
    re = pd.DataFrame(ddict.items())
    re.iloc[:,-1] = re.iloc[:,-1].astype('int32')
    re.columns = ['Tools','Counts']
    return re


In [ ]:
def counts(df):
    
    """ 
    Combines all columns in dataframe.
    Strip Whitespaces from from both edges and also removes internal whitespaces.
    then Counts words from string.
    
    return Dataframe.
    """
    # Joining the columns as one
    new_df = df.where(df.ne('nan')).fillna('').agg(' '.join, axis = 1).to_frame('Tools')
    
    # Removing Whitespaces from Edges
    new_df['Tools'] = new_df['Tools'].str.strip()
    
    # Replacing internal whitespaces with ','
    new_df['Tools'] = new_df['Tools'].apply(lambda x: re.sub('\s+', ', ', x))
    
    # Replacing '' with 'None'
    new_df['Tools'] = new_df['Tools'].replace('', 'None')
    
    # Counting strings
    new_df['Counts'] = new_df['Tools'].apply(lambda n: len(n.split(', ')))
    
    # Droping None values from "Feature"
    new_df = new_df[new_df['Tools']!= 'None']
    
    return new_df


In [ ]:
def RemoveSpaceBetStrings(df):
    
    """
    Replace whitespaces between two words with "_"
    """
    for col in df.columns:
        df[col] = df[col].str.strip().str.replace(" / ", "_")
        df[col] = df[col].str.strip().str.replace(" ", "_")
    
    return df

## **Q1 - What is your age group?**

In [ ]:
age_val = round(df['Q1'].value_counts(normalize= False)*100,2).reset_index().rename(
                                    index=str,columns={'index': 'Age_Group', 'Q1':'Frequency'}
).sort_values(by = 'Age_Group')

# Age Group Pie Chart
fig = px.pie(age_val, values='Frequency', names='Age_Group', title='Kagglian Age Group Distribution')
fig.show()

**Facts -**

- **Around 56%** kagglians are belongs to **Under 30** age.
- **around 23%** kagglians are belong to **Under 40** age.


**Observations -**

- Most of Kagglians are either college students or early mid-age participants.
- We also has some participants from **Above 50** years age also, they might be exicited to learn new things.

## **Q2 - What is your Gender?**

In [ ]:
gen = round(df['Q2'].value_counts(normalize= False)*100,2).reset_index().rename(
                                    index=str,columns={'index': 'Gender', 'Q2':'Frequency'})#.sort_values(by = 'Age_Group')

# Gender Distribution Pie Chart
fig = px.pie(gen, values='Frequency', names='Gender', title='Kagglian Gender Status')
fig.show()

**Facts -**

- **79%** kagglian are **Man**, while only **19** kagglian are **Woman**.


**Observation -**
- Looks like we have more **Man** participants in kaggles as compare to their counter part.
- **Woman** participants may be lacking or hesititate to participats.
- Their also some participants who perfer not to disclouse their gender are consists **2%**.

## **Q3 - Which country do you currently reside?**

In [ ]:
country = round(df['Q3'].value_counts(normalize= True)*100,2).reset_index().rename(
                                    index=str,columns={'index': 'Country', 'Q3':'%'})#.sort_values(by = 'Age_Group')

country['Country'] = country['Country'].replace(['Iran, Islamic Republic of...', 'United Kingdom of Great Britain and Northern Ireland', 'Viet Nam']
                                                ,['Iran', 'UK', 'Vietnam', ])

In [ ]:
fig = px.bar(country, y='%', x='Country', title="Kagglian's Country")
fig.show()


**Facts -**


- **Around 29%** kagglian are from only from **India**.
- followed by **United States of Nation** with **10%**.


**Observations -**
- Indian contributes more in kaggle as compared to other countries.


### **Continent -**

In [ ]:
q3 = pd.DataFrame(df['Q3']).rename(columns= {"Q3": "Country"})

q3['Continent'] = q3["Country"] # Making Copy of Country Column as Continent

In [ ]:
# Replacing Countries with Continents

# #Asia
q3['Continent'] = q3['Continent'].replace(
    to_replace=['India','Japan', 'China', 'Pakistan','Indonesia', 'Turkey','South Korea','Taiwan', 'Israel','Bangladesh'
                ,'Viet Nam','Singapore','Thailand', 'United Arab Emirates', 'Iran, Islamic Republic of...','Malaysia','Philippines','Sri Lanka',
                'Saudi Arabia','Hong Kong (S.A.R.)', 'Nepal','Kazakhstan','Iraq']
    ,value = 'Asia')

# #North America
q3['Continent'] = q3['Continent'].replace(
    to_replace=['United States of America', 'Canada','Mexico']
    ,value = 'North_America')

# #South America
q3['Continent'] = q3['Continent'].replace(
    to_replace=['Colombia','Argentina','Peru', 'Chile', 'Ecuador', 'Brazil']
    ,value = 'South_America')

# #Africa
q3['Continent'] = q3['Continent'].replace(
    to_replace=['Nigeria', 'Kenya', 'South Africa', 'Morocco', 'Tunisia', 'Ghana','Uganda','Algeria', 'Ethiopia', 'Egypt']
    ,value = 'Africa')

# #Europe
q3['Continent'] = q3['Continent'].replace(
    to_replace=['Russia', 'United Kingdom of Great Britain and Northern Ireland','Germany', 'Spain','Italy','Poland', 'Ukraine','Netherlands','Portugal'
                ,'Greece','Ireland','Sweden', 'Switzerland','Belgium','Czech Republic', 'Romania', 'Austria'
                ,'Belarus','Denmark', 'Norway', 'France'] 
    ,value = 'Europe')

# Replacing 'Other', 'I do not wish to disclose my location' with "Other"
q3['Continent'] = q3['Continent'].replace(
    to_replace=['Other', 'I do not wish to disclose my location'] 
    ,value = 'Other')

In [ ]:
# Creating DataFrame for value counts
val = q3['Continent'].value_counts(normalize=False).reset_index().rename(
                                    index=str,columns={'index': 'Continent', 'Continent':'Frequency'})

fig = px.pie(val, values='Frequency', names='Continent', title="Continent-wise Kagglian")
fig.show()

**Facts -**

- **51%** kagglian are only from **Asia**.
- where **Europe**, **North America** and **Africa** contributes around **16.7%**, **12.6%**, and **8%** respectively

**Observation -**
- Asian participants more active and frequently using kaggle as compare to other continents participants.

## **Q4 - What is the highest level of formal education do you have?**

In [ ]:
q4 = pd.DataFrame(df['Q4'])
q4['Q4'].unique()

In [ ]:
# Replacing Some values
rep = {
   'Some college/university study without earning a bachelor’s degree': 'Non-Bachelor’s degree'
    , 'Professional doctorate' : 'Doctoral degree'
    ,'I prefer not to answer':'None/HighScool'
    , 'No formal education past high school' : 'None/HighScool'
}

q4['Q4'] = q4['Q4'].replace(rep)

In [ ]:
# Creating Dataframe for Value counts
edu_val = q4['Q4'].value_counts(normalize=False, dropna=False)\
.reset_index()\
.rename(index = str, columns = {'index': 'Education Level', 'Q4':'Frequency'})


fig = px.pie(edu_val, values= 'Frequency', names = 'Education Level', title = "Kagglian's Educational Level")
fig.show()

**Facts -**

- **39%** of Kagglian are holding Master's Degree.
- while **38%** are holding Bachelor's Degree and **12%** are  holding Doctoral Degree.


**Observation -**
**Master's Degree** holders participats more frequent as compared with other degrees. Followed by **Bachelor's Degree** and **Doctoral's Degree**. 

## **Q5**

In [ ]:
df['Q5'].unique()

In [ ]:
val = df['Q5'].value_counts(normalize = True, dropna = False)\
.reset_index()\
.rename(index = str, columns = {'index': 'Professional Level', 'Q5':'%'})

val['%'] = val['%']*100


fig = px.bar(val, y = 'Professional Level', x = '%', title = "Kagglian's Profession")
fig.show()

**Observations -**

- **Students** are more active on Kaggle than other profession with **around 26%**.
- followed by **Data Scientist** with **around 14%**.


- We can say that, on Kaggle **Students** and **Data Scientists** are more active and frequently contributing towards community.

## **Q6**

In [ ]:
code_years = df['Q6'].value_counts(normalize = True, dropna = False)\
.reset_index()\
.rename(index = str, columns = {'index' : 'Years_Coding', 'Q6':'Percentage'})

code_years['Percentage'] = code_years['Percentage']*100

In [ ]:
fig = px.bar(code_years, x = 'Percentage', y = 'Years_Coding', title = "Years of Coding of Participant")
fig.show()

Most of coders are either **1-3 years** or **< 1 years** of coding experience. Both of it consists **around 53%** of all participants. We can say thses participants are either students or learners too.


We can also see some participants **Never Coded** are about to **4%**.

## **Q7 - Which Programming Languages you used on regular basis?**

#### **Total Programing Languages used regularly per Participants**

In [ ]:
# Question 7 - Programming Languages used Regular Basis.
q7 = df.iloc[:, 7:20].drop('Q7_Part_12', axis = 1)

pro_languages = counts(q7).rename(columns = {'Tools': 'No_Programming_Lang'})

# All Counts "more than 6" are replaced by 6 and considered as "Programming Languages used more than 6"
pro_languages['Counts'] = pro_languages['Counts'].replace([6,  7,  8,  9, 11, 10, 12], 6)

In [ ]:
# pro_lang = pro_languages[pro_languages['Programing_Languages']!='None']
# # pro_lang['Programing_Languages'].value_counts() 

lang_val = pro_languages['Counts'].value_counts().reset_index()\
.rename(index = str ,columns = {'index': 'No_Programming_Lang', 'Count_Programing_Languages' :'Counts'})\


In [ ]:
fig = px.bar(lang_val, x = 'No_Programming_Lang', y = 'Counts'
             ,title = 'Number of Programming Languages used Regularly')

fig.show()

**Facts -**
- **2 programming languages** are used regularly by **7000** participants,
- **1 programming language** is used regularly by **6000** participants,
- **3 programming languages** are used regularly by **5600** participants. 



Most data scienties are use 1 to 3 different languages on regular basis.


In [ ]:
# Programing Languages Regularly Used by Participants
pro_lang = clean(data,'Q7_Part_',12).sort_values(by = 'Counts').rename(columns = {"Tools":"Programing Languages"})

In [ ]:
fig = px.bar(pro_lang, x = 'Counts', y = 'Programing Languages', title = 'Proramming Languages used Regularly')
fig.show()

- **Python** is most regularly used language followed by **SQL**.
- **Julia** and **Swift** are less regularly used languages.

- 

## **Q8**

In [ ]:
df['Q8'].unique()

In [ ]:
q8 = df['Q8'].value_counts(normalize = False).reset_index()\
.rename(index = str, columns = {'index':'Programming Languages', 'Q8' : 'Frequency'})


fig = px.bar(q8, x = 'Frequency', y='Programming Languages', title = 'Languages used for Data Science')
fig.show()

- **Python** is widely used programming language for Data Science purpose
- Followed by **R** and **SQL** are also used for data science.

## **Q9**

In [ ]:
q9_vis = clean(df, 'Q9_Part_', 12).sort_values(by = 'Counts', ascending = False).rename(columns = {'Tools':'IDEs'})

In [ ]:
q9_vis['IDEs'] = q9_vis['IDEs'].str.strip()
q9_vis['IDEs'].unique()

In [ ]:
replacer = {
    'Jupyter (JupyterLab, Jupyter Notebooks, etc)' : 'Jupyter'
    ,'Jupyter Notebook':'Jupyter'
    ,'Visual Studio Code (VSCode)': 'VSCode'
    ,'Visual Studio':'VSCode'
}

q9_vis['IDEs'] = q9_vis['IDEs'].replace(replacer)

q9_vis['IDEs'].unique()

In [ ]:
fig = px.bar(q9_vis, x = 'Counts', y = 'IDEs', title = 'Recommened IDEs for Data Science')
fig.show()

- **Jupyter Nootbook** and **Jupyter** are recommended by around **21,700** participants
- followed by **Visual Studio Code** and **Visual Studio** are recommended by around **14,000** participants.
- followed by **PyCharm** also recommended by around **7,500** participants


- These two IDEs, i.e., **Jupyter** and **VS Code** are looks most popular in Data Science, followed by **PyCharm**.

#### **Total Integreated Development Environments used regularly per Participants**

In [ ]:
# Question 9 - Recommened IDE for Data Science.
q9 = df.iloc[:, 21:34].drop("Q9_Part_12", axis = 1)

In [ ]:
# Combining Similar value Columns
q9['Q9_Part_1+11'] = q9[['Q9_Part_1','Q9_Part_11']].where(q9[['Q9_Part_1','Q9_Part_11']].ne('nan')
                                        ).fillna('').agg(' '.join, axis = 1)

q9['Q9_Part_3+4'] = q9[['Q9_Part_3','Q9_Part_4']].where(q9[['Q9_Part_3','Q9_Part_4']].ne('nan')
                                        ).fillna('').agg(' '.join, axis = 1)

q9 = q9.drop(['Q9_Part_1', 'Q9_Part_3', 'Q9_Part_4', 'Q9_Part_11'], axis = 1)

In [ ]:
# Replacing Some similar Values

q9['Q9_Part_1+11'] = q9['Q9_Part_1+11'].replace(to_replace=[
    ' Jupyter Notebook',
    'Jupyter (JupyterLab, Jupyter Notebooks, etc) Jupyter Notebook',
    'Jupyter (JupyterLab, Jupyter Notebooks, etc) ']
                                                , value= 'Jupyter')


q9['Q9_Part_3+4'] = q9['Q9_Part_3+4'].replace(to_replace=['Visual Studio Visual Studio Code (VSCode)'
                                                          ,' Visual Studio Code (VSCode)', 'Visual Studio ']
                                              , value= 'VSCode')

q9['Q9_Part_9'] = q9['Q9_Part_9'].replace('Vim / Emacs', 'Vim_Emacs')

In [ ]:
q9['Q9_Part_1+11'].unique()

In [ ]:
ide_recomm = counts(q9).rename(columns = {'Tools' : 'Recommened IDEs for Data Science'})
ide_recomm

In [ ]:
val = ide_recomm['Counts'].value_counts().reset_index()\
.rename(index = str,columns= {'index': 'No_IDEs'})\
.sort_values(by = 'No_IDEs')

val

In [ ]:
fig = px.pie(val, values = 'Counts', names = 'No_IDEs', title = 'Number of IDEs Used by Participants')
fig.show()

- **30.5%** recommend **1 IDE**, followed by **2** and **3** with **29.3%** and **21.2%** respectively.


**81%** participants uses 1 to 3 IDEs. And remaining uses more than 3 IDEs.

## **Q10** - **Host Notebooks used on Regular Basis**

In [ ]:
q10_com = clean(df, 'Q10_Part_', 17).rename(columns = {'Tools':'Host Notebooks'})\
.sort_values(by = 'Counts', ascending=False)\
.reset_index()\
.drop('index', axis = 1)


q10_com

In [ ]:
fig = px.bar(q10_com
             ,x = 'Counts'
             ,y = 'Host Notebooks'
             ,title = 'Hosted Notebook used Regularly')

fig.show()

- **Colab Notebooks** is used by **around 9,750** participants regularly.
- Followed by **Kaggle Notebooks** is used by **around 9,500** participants regulary.


As we can see above Bar Chart, we can conclude that **around 19,000** participants frequently using either **Colab Notebooks** or **Kaggle Notebooks** due to these host notebooks might has pre-installed dependencies and faster than other host notebooks.

#### **Total Host Notebooks used regularly per Participants**

In [ ]:
q10 = df.iloc[:, 34:51]

In [ ]:
RemoveSpaceBetStrings(q10) 

In [ ]:
q10['Q10_Part_4'].unique()

In [ ]:
freq = counts(q10)

freq['Counts'] = freq['Counts'].replace([6,7,8,9,10,11,14,15,16], 6) # Hosted Notebooks Used 6 and more than 6

In [ ]:
val = freq['Counts'].value_counts().reset_index().rename(columns = {'index' : 'No of Hosted Notebooks'
                                                                    })
val

In [ ]:
fig = px.bar(val, x = 'No of Hosted Notebooks', y = 'Counts',title = "Number of Hosted Notebooks Used Regularly")
fig.show()

**Facts -**

- **atleast 1** hosted notebook are used regularly by **more than 9000** data scientist.
- **2** hosted notbook are used regularly by **around 5750** data scientis.

In data science upto 2 hosted notebooks are used regularly by data scientiest. These hosted notebooks might have good speed and may has pre-install dependencies, so during work data scientiest doesn't have to install required dependecies. Most data scientists uses **2 Hosted Notebooks** are either **Colab** or **Kaggle Notebook**. These 2 are widely used.

# **Q11**

In [ ]:
workstation = df['Q11'].value_counts().reset_index().rename(columns = {'index': 'Workstations', 'Q11':'Counts'})
workstation

In [ ]:
fig = px.pie(workstation, values='Counts', names = 'Workstations', title = 'Workstations used for Data Science Purpose')
fig.show()

**Facts -**

- Most data scientiest has **a Laptop** as **1st** preferance as workstations **65.7%** .
- followed by **a Personal Computer / Desktop** as another mean of workstations **along with 19.9%**.
- **Cloud Computing Platforms** as **3rd** preference with **9.42%**


**A Laptop** and **A Personal Computer / Desktop** both are favorite workstations for Data Scientiest as well as **Cloud Computing Platform** also used but those are chargeable. 

A **Deep Learning Workstations** are also used but not to solve regular problem solving but in **Deep Learning** or **Advance Deep Learning** problems for large dataset or large featured datasets.

# **Q12**

**`GPUs` Used Regular Basis**

In [ ]:
q12 = df.iloc[:, 52:58]
q12

In [ ]:
gpus = counts(q12)
gpus

In [ ]:
val = clean(df, 'Q12_Part_', 6).rename(columns = {'Tools':'DL_Hardware', 'values':'Counts'})
val.sort_values(by = 'Counts',ascending=False)

In [ ]:
fig = px.bar(val, x = 'DL_Hardware', y = 'Counts', title = 'Special Type of Hardware used Regularly')
fig.show()

**Facts -**

- **Around 13000** Data scientists used **None** Deep learning Hardwares, which accumulate **around 45%**
- **8000** Data Scientiests uses **NVIDIA GPUs** .
- **around 3750** data scientiest uses **Google Cloud TPUs**.


**NVIDIA GPUs** are most famouse and highly used and recommended by data scientiests and data science communities too. And it is bulid-in with PCs / Laptops

And for using **Google Cloud TPUs**, **AWS Trainium Chips** and **AWS Inferentia Chips** are cloud based GPUs / TPUs. And less popular for Data Science.

# **Q13**

**`TPU` Uses**

In [ ]:
q13_val = df['Q13'].value_counts().reset_index().rename(index = str, columns = {'index' : 'TPU Used', 'Q13' :'Counts'})
q13_val

In [ ]:
fig = px.pie(q13_val, values='Counts', names = 'TPU Used', title = 'TPU Used Times')
fig.show()

**Facts -**

- **67.4%** participants has **NEVER** used TPUs.
- while **14%** participants has used TPUs **2-5 times**.
- and **12.2%** participants has used TPU only **ONCE**.


**Observation -**
- Most participants prefer **not to depend on TPUs** or they don't know how to use TPU in particalur projects.

# **Q14**

**Which `Data Visualization` tool or libraries used on regular basis**

In [ ]:
data_viz_packages = clean(df, 'Q14_Part_', 12).rename(columns = {'Tools':'Visualization Libraries','values':'Counts'})
data_viz_packages

In [ ]:
fig = px.bar(data_viz_packages, x = 'Visualization Libraries', y = 'Counts', 
             title = "Data Visualization Tools Used Regularly")

fig.show()

**Facts -**

- **around 17,500** participants uses **Matplotlib**.
- followed by **Seaborn** and **Plotly/Plotly Express** with around **12,500** and **5750** participants respectively.


**Observations -**

As usual **Matplotlib** are most favourite data visualization tools in Data Science field. Other most favourite visualization tools are **Seaborn** and **Plotly**. These visualization tools are easy to uses and faster to visualize the data.



### **Total Visualization Libraries used regularly per Participants**

In [ ]:
q14 = df.iloc[:, 59:71].drop('Q14_Part_11', axis = 1)
q14.head(3)

In [ ]:
# Replace '/' with "_"

q14['Q14_Part_3'] = q14['Q14_Part_3'].replace(' Plotly / Plotly Express ', 'Plotly(PX)')
q14['Q14_Part_4'] = q14['Q14_Part_4'].replace(' Ggplot / ggplot2 ', 'Ggplot(ggplot2)')
q14['Q14_Part_10'] = q14['Q14_Part_10'].replace(' Leaflet / Folium ', 'Leaflet/Folium')

In [ ]:
count_val = counts(q14).rename(columns = {'Feature': 'Visualization Tools'})
count_val

In [ ]:
plt.figure(figsize=(9, 7))
sns.countplot(count_val['Counts'])
plt.xlabel('Number of Data Visualization Tool Counts')
plt.title("Number of Data Visualization Tools used by Participants", )

**Facts -**

- **around 7500** participants uses **2** visualization tools at a time.
- **around 5500** participants uses only **1** visualization tool at a time.
- **around 4400** participants uses **3** visualization tools at a time.


**Observations -**

Participants likes to use 1 to 3 data visualization tools / libraries to visualize data at same time.

# **Q15**

**How many years you are using `Machine Learning` methods?**

In [ ]:
df['Q15'].value_counts()

In [ ]:
df['Q15'].value_counts().plot.barh(figsize = (8, 7), title = 'Since When ML Methods are Used ?')

**Facts -**

- **more than 9000** participants are using ML methods are lies in **Under 1 year**
- **around 4700** participants are using ML methods are lies in **1-2 years**.
- **around 3900** participants **Never used ML methods** but they might starting or planning to use ML methods now onwards.


**Observation -**

Those who are using ML methods **more than 2 years** are *Less active on Kaggle*. But those who are using ML methods **upto 2 years** are *Very active on Kaggle*. Even some participants never used ML methods before those are also active on Kaggle and willing to use ML methods in futures.

# **Q16** 

**Which `Machine Learning Frameworks` do you use on a regular basis?**

In [ ]:
ml_frame = clean(df, 'Q16_Part_', 18).rename(columns = {'Tools' : 'ML Frameworks'
                                             ,'values':'Counts'}).sort_values(by = 'Counts', ascending = False)

ml_frame

In [ ]:
px.bar(ml_frame, x = 'ML Frameworks', y = 'Counts', title = 'Machine Learning Fremeworks used Regulary by Participants')

Looks **Scikit-Learn** is most favourite and popular ML framework among other frameworks. **Around 14000** participants uses scikit-learn framework for ML. Followed by **TensorFlow** and **Keras** these two frameworks used in advance ML application, i.e., **`Deep Learning`**, with around **9300** and **8000** participants  respectively.

Their are also some other ML frameworks such as **PyTorch** and **XGBoost** with **around 6000** participants in each frameworks.

These 5 frameworks are used frequently in Machine Learning.

### **Total Machine Learning Frameworks used regularly per Participants**

In [ ]:
q16 = df.iloc[:, 72:90].drop('Q16_Part_17', axis = 1) # Dropping 'Q16_Part_17' because it hase "None" value
q16

In [ ]:
ml_pac = counts(q16)
ml_pac

In [ ]:
ml_pac['Counts'].value_counts().plot.bar(figsize = (8, 7), title = 'No. of ML Frameworks used Regulary')

**Facts -**

- Most frequently **1** machine learning fremework is used by around **4500** participants.
- followed by **3** and **2** machine learning fremeworks are used by around **3600** and **3500** participants respectively.

**Observations -**

Most participants uses **1** ML fremework to **4** ML fremeworks on regular basis.

# **Q17**

**Which `ML Algorithms` do you use on a regular basis?**

In [ ]:
ml_algo = clean(df, "Q17_Part_", 12).rename(columns = {'Tools':"ML Algorithms"})\
.sort_values(by = 'Counts', ascending = False)

ml_algo

In [ ]:
fig = px.bar(ml_algo, x = 'Counts', y = 'ML Algorithms', title = 'Machine Learning Algorithm used Regularly')
fig.show()

**Facts -**

- **Upto 13900** participants uses **Linear or Logistic Regression** regularly.
- **Upto 11900** participants uses **Decision Trees or Random Forests** regularly.
- **Gradient Boosting Machines** and **Convolutional Neaural Networks** are also used regularly by around **7500** and **7400** respectively.


**Observation -**

> - **Linear or Logistic Regression** and **Decision Tree or Random Forests** are highly popular Machine Learning algorithms and widely used algorithms in solving Supervised Machine Learning problems. These algorithms are fast and less complicated.
> - **Gradient Boosting Algorithms** also used to provide boost to base estimator algorithms. **CNN** [Convolutional Neaural Networks] is used more frequently in Deep Learning problems.
> - We can see **Supervised Machine Learning** algorithms are used more frequently than **Unsupervised Machine Learning** algorithms. And these algorithms also very popular not only in Kaggle but also in solving Real Time problem. 

### **Total Machine Learning Algorithms used regularly per Participants**

In [ ]:
q17 = df.iloc[:, 90:102].drop('Q17_Part_11', axis = 1)
q17.head(3)

In [ ]:
# Replace ' ' with '_' between Strings

q17['Q17_Part_1'] = q17['Q17_Part_1'].replace('Linear or Logistic Regression', 'Linear/Logistic_Regression')
q17['Q17_Part_2'] = q17['Q17_Part_2'].replace('Decision Trees or Random Forests', 'Decision_Trees/Random_Forests')
q17['Q17_Part_3'] = q17['Q17_Part_3'].replace('Gradient Boosting Machines (xgboost, lightgbm, etc)'
                                              ,'Gradient_Boosting_Machines(xgboost_lightgbm_etc)')

q17['Q17_Part_4'] = q17['Q17_Part_4'].replace('Bayesian Approaches','Bayesian_Approaches')
q17['Q17_Part_5'] = q17['Q17_Part_5'].replace('Evolutionary Approaches', 'Evolutionary_Approaches')
q17['Q17_Part_6'] = q17['Q17_Part_6'].replace('Dense Neural Networks (MLPs, etc)'
                                              ,'Dense_Neural_Networks(MLPs_etc)')

q17['Q17_Part_7'] = q17['Q17_Part_7'].replace('Convolutional Neural Networks','Convolutional_Neural_Networks')
q17['Q17_Part_8'] = q17['Q17_Part_8'].replace('Generative Adversarial Networks','Generative_Adversarial_Networks')
q17['Q17_Part_9'] = q17['Q17_Part_9'].replace('Recurrent Neural Networks','Recurrent_Neural_Networks')
q17['Q17_Part_10'] = q17['Q17_Part_10'].replace('Transformer Networks (BERT, gpt-3, etc)'
                                                ,'Transformer_Networks(BERT_gpt-3_etc)')

In [ ]:
ml_algo_counts  = counts(q17).rename(columns = {'Feature':"Machine Learning Algorithms"})
ml_algo_counts


In [ ]:
ml_algo_counts['Counts'].value_counts().plot.bar(figsize = (10,7), title = 'No. of Machine Learning Algo. Used Regularly')
plt.xlabel('No. Of ML Algorithms')
plt.ylabel('Frequency')

**Facts -**

- **3 ML Algorithms** are used regulary by around **4500** participants.
- follopwed by **2** and **1 ML Algorithms** are used regularly by around **3750** and **2900** participants respectively.
- **4 ML Algorithms** used regularly by around **2750** participants.


**Observations -**

**3 ML** algorithms are used mostly by participants and followed by **2**, **1** and **4** algorithms are also used. We can say, in most Machine Learning problems upto **4 ML Algorithms** are used in real time or those are the final Machine Learning algorithms.

# **Q18**

**Which categories of computer vision methods do you use on a regular basis ?**

In [ ]:
cv_methods = clean(df, 'Q18_Part_', 7)
cv_methods['Tools'] = cv_methods['Tools']\
.replace('Image classification and other general purpose networks (VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)'
         ,'Image classification and other(VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)') 

In [ ]:
cv_methods['Tools'].unique()

In [ ]:
fig = px.bar(cv_methods, x = 'Counts', y = 'Tools', title = 'Computer Vision Methods used Regularly')
fig.show()

**Facts -**

- **Image Classification and Other** Computer vision methods are used regularly by **around 4250** data Scientiests.
- followed by **Image Segementation** and **Object Detection** methods are used frequently by **around 2700** each.
- **General Purpose image / video tools** are also used by **below 2700**.



**Image Classifiaction and Other** and **Image Segmentation** methods are used more frequently and widly used computer vision method. Followed by **Object Detection** methods are also used regularly.

### **Total Computer Vision Methods used regularly per Participants**

In [ ]:
q18 = df.iloc[:, 102:109].drop('Q18_Part_6', axis = 1)
q18.head()

In [ ]:
q18['Q18_Part_5'].unique()

In [ ]:
q18['Q18_Part_1'] = q18['Q18_Part_1'].replace('General purpose image/video tools (PIL, cv2, skimage, etc)'
                                              ,'General_purpose_image/video_tools(PIL/cv2/skimage/etc.)')


q18['Q18_Part_2'] = q18['Q18_Part_2'].replace('Image segmentation methods (U-Net, Mask R-CNN, etc)'
                                              ,'Image_segmentation_methods(U-Net/Mask_R-CNN/etc.)')

q18['Q18_Part_3'] = q18['Q18_Part_3'].replace('Object detection methods (YOLOv3, RetinaNet, etc)'
                                              ,'Object_detection_methods(YOLOv3_RetinaNet/etc.)')

q18['Q18_Part_4'] = q18['Q18_Part_4']\
.replace('Image classification and other general purpose networks (VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)'
         ,'Image_classification_and_other_general_purpose_networks(VGG/Inception/ResNet/ResNeXt/NASNet/EfficientNet/etc.)')
                                              
q18['Q18_Part_5'] = q18['Q18_Part_5'].replace('Generative Networks (GAN, VAE, etc)', 'Generative_Networks(GAN/VAE/etc.)')


In [ ]:
cnt = counts(q18)
cnt.head()

In [ ]:
cnt['Counts'].value_counts(normalize=True).plot(kind = 'barh', figsize = (9, 7))
plt.title("No. of Computer Vision Methods used Regularly")
plt.xlabel('No. of CV Methods')
plt.ylabel('Counts')


**Facts -**

- **1** computer vision methods are used **around 32%** data scientists on regular basis.
- **2** computer vision methods are used **around 29%** data scientists on regular basis.
- **3** computer vision methods are used **around 21%** data scientists on regular basis.


**Upto 82%** Data Scientists uses either **1** or **2** or **3** Computer vision methods on regular basis.It might be either one or multiple of following -
   - **Image Classification**
   - **Image Segmentation**
   - **Object Detection**
   - **General Purpose Image / Video Tools**
    
   

# **Q19**

**Which Natural Language Processing (NLP) Methods used on a regular basis?**

In [ ]:
nlp_meth = clean(df, 'Q19_Part_', 6).rename(columns = {'Tools':'NLP_methods'})
nlp_meth

In [ ]:
fig = px.pie(nlp_meth, values = 'Counts', names = 'NLP_methods', title = "NLP Methods used Regularly")
fig.show()

**Facts -**

- **Word-Embeddings/Vectors** NLP method is used by **22.6%** data scientists on regular.
- **Transformer Language Models** NLP method is used by **20.1%** data scientists on regular.
- **Encoder-Decoder Models** NLP method is used by **17.3%** data scientists on regular.
- **Other** NLP method is used by **22.1%** data scientists on regular.


**Word-Embeddings/Vectors**, **Transformer Language Models** and **Encoder-Decoder Models** are highly used Natural Language Processing methods which are used regularly by many data scientists.

### **Total NLP Methods used regularly per Participants**

In [ ]:
q19 = df.iloc[:, 109:115].drop('Q19_Part_5', axis = 1)
q19

In [ ]:
# Replace " " with "_" in strings

q19['Q19_Part_1'] = q19['Q19_Part_1'].replace('Word embeddings/vectors (GLoVe, fastText, word2vec)'
                                              ,'Word_Embeddings/Vectors')

q19['Q19_Part_2'] = q19['Q19_Part_2'].replace('Encoder-decorder models (seq2seq, vanilla transformers)'
                                              ,'Encoder-Decorder_Models')
q19['Q19_Part_3'] = q19['Q19_Part_3'].replace('Contextualized embeddings (ELMo, CoVe)', 'Contextualized_Embeddings')
q19['Q19_Part_4'] = q19['Q19_Part_4'].replace('Transformer language models (GPT-3, BERT, XLnet, etc)'
                                              ,'Transformer_Language_Models')

In [ ]:
count_q19 = counts(q19).rename(columns = {'Tools':"No_NLP_methods"})
count_q19.head()

In [ ]:
count_q19['Counts'].value_counts(normalize = True).plot.bar(figsize = (9, 5), title = 'No. of NLP Methods Used Regularly')
plt.xlabel('No. of NLP Methods Used')
plt.ylabel("Counts")

**Facts -**

- **around 36%** data scientists are using any **1** NLP method regularly.
- **around 34%** data scientists are using any **2** NLP methods regularly.
- **around 20%** data scientists are using any **3** NLP methods regularly.


**Around 90%** data scientists are using any **1** to **3** NLP methods on regular basis. Either one or multiple out-off following -
  
  - Image Classification
  - Image Segmentation
  - Object Detection
  - Genral Purpose Images / Videos

# **Q20** 

**In which industry your current employer/contract works**

In [ ]:
df['Q20'].value_counts()

In [ ]:
df['Q20'].value_counts(normalize=True).plot.barh(figsize = (12,8), title = "Current Employer's Industries")

**Facts -**

- **25%** participants are employed in **Computers/Technology** Industry.
- **19.50%** participants are employed in **Academics/Education** Industry.
- **9%** participants are employed in **Accounting / Finance** Industry.


- Participants from **Hospitality/Entertainments/Sports**, **Military/Security/Defence**, **Online Business/Internet-based Sales** and **Non-Profit/Service** Industries are employed **Less than 2%** data scientists.

# **Q21**

**What is the size of the company where you are employed?**

In [ ]:
df['Q21'].value_counts()

In [ ]:
df['Q21'].value_counts(normalize=True).plot.barh(figsize = (9, 5), title = "Size of the Company")
plt.xlabel('percentages')
plt.ylabel("Size of the Company")

**Facts -**


- **31%** participants are from **0-49 employees** companies.
- **21%** participants are from **10,000 or more employees** companies
- **19%** participants are from **1000-9,999 employees** companies


Most participants are from **Start-up Companies** followed by **Large Companies or MNCs**. We can say that, Start-Up Companies and MNCs are using Data Science more frequently rather other companies. 

# **Q22**

**Approx. how many individuals are responsible for data science workloads at your place of business?**

In [ ]:
df['Q22'].value_counts(normalize=True)

In [ ]:
df['Q22'].value_counts(normalize=True).plot.barh(figsize = (9, 5), title = 'No. of Individuals Responsible for DS Works')
plt.xlabel("percentage")
plt.ylabel('Size of DS Workloads')

**Facts -**

- **22.5%** companies has **1-2** individuals whose are responsible for Data science workloads.
- **22%** companies has **20+** individuals whose are responsible for Data science workloads.
- **19%** companies has **0** individual responsible for Data science workloads, 
    - **Either these companies not has Data Science Department or these companies don't want to disclose this.**
    
    
- **16%** companies has **3-4** individuals whose are responsible for data science workloads.



In **38.5%** companies might has **1 to 4** individuals, whose are actually responsible for complate data science workloads. These companies are either **Small Companies** or **Start-Up Companies**. While where **20+** individuals are responsible for data science workloads are might be **Large Companies or MNCs**.



# **Q23**

**Does your current employer incorporate machine learning methods into their business?**

In [ ]:
# Replacing Values with Small Values

rep = {
    'No (we do not use ML methods)' : 'No, Do not use ML'
    
    ,'We are exploring ML methods (and may one day put a model into production)' : "Exploring ML Methods"
    
    ,'We use ML methods for generating insights (but do not put working models into production)':\
    "Using ML Methods for Insights(not production)"
    
    ,'We have well established ML methods (i.e., models in production for more than 2 years)' :\
    "Established ML Methods for > 2years(production)"
    
    ,'We recently started using ML methods (i.e., models in production for less than 2 years)':\
    "Started ML Methods fro < 2years(production)"
}

df["Q23"] = df["Q23"].replace(rep)

In [ ]:
df['Q23'].value_counts(normalize=True,dropna=False)

In [ ]:
df['Q23'].value_counts(normalize=True,dropna=False).plot.barh(figsize = (9, 5))
plt.title("Current Employer Incorporate ML Methods")
plt.xlabel("percentage")
plt.ylabel("Incorporate ML Methods")

**Facts -**

- **13%** companies are just **Exploring ML Methods**.
- **12.5%** companies are **Not Using ML Methods**.
- **around 25%** companies are **Using ML Methods** are either one  of -
   - **Established and using more than 2 years in productions** or
   - **Started and using less than 2 years in productions** or
   - **Using for Understanding Patterns**
   
   
 - **around 39%** has Missing values.

# **Q24**

**What activities that makes an important part of your role at work?**

In [ ]:
df['Q24_Part_1'] = df['Q24_Part_1'].replace('Analyze and understand data to influence product or business decisions'
                                            ,'Analyze_and_Understand_Data')

df['Q24_Part_2'] = df['Q24_Part_2']\
.replace('Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data'
        ,'Build_and/or_Run_the_Data_Infrastructure')

df['Q24_Part_3'] = df['Q24_Part_3'].replace('Build prototypes to explore applying machine learning to new areas'
                                            ,'Build_Prototypes')

df['Q24_Part_4'] = df['Q24_Part_4']\
.replace('Build and/or run a machine learning service that operationally improves my product or workflows'
         ,'Build_and/or_run_ML_service')

df['Q24_Part_6'] = df['Q24_Part_6'].replace('Do research that advances the state of the art of machine learning'
                                            ,'Research_to_advances_ML')

df['Q24_Part_5'] = df['Q24_Part_5'].replace('Experimentation and iteration to improve existing ML models'
                                            ,'Improve_Existing_ML_Models')


df['Q24_Part_7'] = df['Q24_Part_7'].replace('None of these activities are an important part of my role at work'
                                            ,'None_of_these_activities')


In [ ]:
q24 = df.iloc[:, 119:127].drop('Q24_Part_7', axis = 1)
q24

In [ ]:
imp_tasks = clean(df, 'Q24_Part_', 8).rename(columns = {'Tools' : 'Imp. Tasks as part of Works'})
imp_tasks

In [ ]:
fig = px.bar(imp_tasks
             ,x = 'Counts'
             ,y = 'Imp. Tasks as part of Works'
             ,title = 'Important Tasks as Part of Works')

fig.show()

**Facts -**

- **9100** individuals works on **Analyze and Understand Data**
- **5100** individuals works on **Building Prototypes for Machine Learning models**
- **4500** individuals works on **Build and/or run the Data Infrastructure**
- **around 4000** individuals works on **Improve existing ML Models**


### **Total Tasks Performed by Participants Regularly**

In [ ]:
tasks = counts(q24)

In [ ]:
tasks.Counts.value_counts(normalize = True).plot.bar(figsize = (9, 5)
                                                    ,title = 'No. of tasks performed')
plt.xlabel('No. of tasks performed in Works')
plt.ylabel('percentage')

**Around 80%** individuals works on either one or more tasks simultaneously for any tasks from below -

- **Analyze and Understand Data**
- **Builduing Prototypes for Machine Learning**
- **Build and/or run the Data Infrastructure**
- **Improve Existing ML Models**
- **Build and/or run ML Services**

## **Q25**

## **Q26**


**Approximately how much money have you (or your team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (approximate USD)?**

In [ ]:
data['Q26'].unique()

In [ ]:
val = df["Q26"].value_counts()

# Pie Chart
fig = px.pie(val
             ,values = val.values
             ,names = val.index
             ,title = "Money Spent on ML / Clould At Home in last 5Years")

fig.show()

**Facts -**

- **5903** participants spend **0 USD** in last 5 years.
- **2534** participants spend **100-999 USD** in last 5 years.
- **2491** participants spend **1000-9999 USD** in last 5 years.
- **1929** participants spend **1-99 USD** in last 5 years.
- **1309** participants spend **10,000 - 99,9999 USD** in last 5 years.
- **1017** participants spend **100,000 or more USD** in last 5 years.


**Observations -**

**39%** participants used those cloud platform services or machine learning at their home which either **Free** or **One year free subscription** in last 5 years.These participants either **College Students** or **New Skill Learners**. 


**17%** participants used those cloud platforms which charges **less than 1000USD** either these participants are **FreeLancers** or **Counsultants**

# **27 - Cloud Computing Platforms**


- **Present** - Q27-A - Which Cloud Computing Platforms do you use on a regular basis?
- **Future** - Q27-B - Which Cloud Computing Platforms do you hope to become more familiar with in the next 2 years?

In [ ]:
data['Q27_A_Part_1'].unique(), data['Q27_B_Part_1'].unique()

In [ ]:
# Q27_A and Q27_B
cld_plt = clean(df, 'Q27_A_Part_', 11).rename(columns = {'Tools':'Cloud Platforms'}).sort_values(by= "Counts"
                                                                                                 ,ascending=False)
cld = clean(df, "Q27_B_Part_", 12).rename(columns = {"Tools":"Cloud_Platforms"}).sort_values(by= "Counts"
                                                                                             ,ascending=False)

# Sub-Bar Charts
title = 'Cloud Computing Platforms: Present-use vs Future-wish'

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing = 0.3,
                    shared_yaxes=False,
                   )

fig.add_trace(go.Bar(y=cld_plt['Cloud Platforms'],
                     x=cld_plt["Counts"],
                     orientation= 'h',
                     name="Present",
                     marker_color='lightseagreen',
                     opacity =0.6
                    ),row=1, col=1,
             )

fig.add_trace(go.Bar(y = cld['Cloud_Platforms'],
                     x = cld['Counts'],
                     orientation= 'h',
                     name="Future",
                     marker_color='gold',
                     opacity =0.6
                    ),row=1, col=2,
             )

fig.update_traces(marker_line_color='black',
              marker_line_width=1.5)

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.2,
                  height=450, width=950,
                  barmode='group',
                  titlefont={'size': 28},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')
                 

fig.update_layout(xaxis_title='Count',yaxis_linewidth=0,
autosize=False,
margin=dict(
    l=210, r=150, b=100, t=70,
),
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

**Facts -**
- in **Presently using Regular Basis**
  - **Amazon Web Services (AWS)** used by **around 3721** participants regularly
  - followed by **Google Cloud Platform (GCP)** and **Microsoft Azure** ysed by around **3142** and **2450** respectively on regular basis.
  - Other cloud service providers also used reglarly as -
    - **IBM Cloud / Red Hat** used by **572**
    - **Oracle Cloud** used by **454**
    - **VMware Cloud** used by **390** and so on.
  
- In **Future** [in next 2 years] -
  - **7494** participants are planing to work in **Amazon Web Services (AWS)**.
  - followed by **Google Cloud Platform (GCP)** with **7484** and **Microsoft Azure** with **5648** participants respectively planning to work in those Cloud Platforms.
  
  

**Observation -**

From **Present Bar Chart** and **Future Bar Charts**, we can see **AWS** is widely used regularly in present as well as 1st priority to work in future in cloud service and it might be having large services and has high demands to. Followed by **GCP** and **Microsoft Azure**.

### **Total Cloud Computing Platforms Use per Participants**

In [ ]:
q27A = df.iloc[:, 129:141].drop('Q27_A_Part_11', axis = 1)
q27B = df.iloc[:, 268:280].drop('Q27_B_Part_11', axis = 1)

# Applying RemoveSpaceBetStrings function to replace internal whitespaces between words
RemoveSpaceBetStrings(q27A)
RemoveSpaceBetStrings(q27B)

# Cloud Platform counts uses by participants
plt_cld = counts(q27A).rename(columns = {"Tools":"Cloud_Platform"})
plt = counts(q27B).rename(columns = {"Tools":"Cloud_Platform"})

In [ ]:
val_A = plt_cld['Counts'].value_counts()
val_B = plt['Counts'].value_counts()

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=val_A.index
                     ,values=val_A.values
                     ,name="Present")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=val_B.index
                     ,values=val_B.values
                     ,name="Future")
              ,row=1
              ,col=2)

fig.update_layout(title="Cloud Platforms Counts : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
              ,template='simple_white'
              ,paper_bgcolor='#F5F5F5'
              ,plot_bgcolor='#F5F5F5'
            ).update_yaxes(categoryorder='total ascending')

**Facts -**
- In **Present**
  - **3892** participants are using any **1** cloud service provider which are **55.3%**
  - **1945** participants are using any **2** cloud service providers which are **27.7%**
  - **827** participants are using any **3** cloud service providers which are **11.8%**
  - **368** particpicants are using **4 or more than 4** cloud services regluarly which are **5.2**.


- In **Future**
   - **3207** participants are planning to use any **1** cloud service provider which are **28.5%**
   - **2998** participants are planning to use any **2** cloud service providers which are **26.6%**
   - **2685** participants are planning to use any **3** cloud service providers which are **23.9%**
   - **1319** participants are planning to use any **4** cloud service providers which are **11.7%**
   - **1047** participants are planning to use **5+** cloud service providers which are **9.3%**


**Observations -**

- **Around 95%** participants are using from **1** to **3** cloud service providers on regular basis. And thse cloud providers are might has ample of services and widely used in Industries too.

- **Around 90%** participants are planning to use from **1** to **4** cloud providers in next **2 years** as their future requirements.

In [ ]:
# import plotly.graph_objects as go

# # pull is given as a fraction of the pie radius
# fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
# fig.show()

In [ ]:
# Creating dataframe using counts from "value_counts" function

# large = plt_cld.groupby("Counts").filter(lambda x: len(x) == 16)
# large

## **Q28 - the Cloud Platforms that you are familiar with, which has the best developer experience?**

In [ ]:
data['Q28'].unique()

In [ ]:
# Stripping Whitespaces
df['Q28'] = df['Q28'].str.strip()

# Checking Changes made or not
df.Q28.unique()

In [ ]:
df['Q28'].isna().sum()

In [ ]:
val_q28 = df['Q28'].value_counts().sort_values()


fig = px.bar(val_q28
             ,y = val_q28.index
             ,x = val_q28.values
             ,title = "Enjoyable Cloud Platforms"
             ,labels={"index" : "Cloud Platforms"
                      ,"x":"frequencies"}
            )
fig.show()

**Facts -**

- **836** participants enjoys **Amazon Web Services**(AWS).
- followed by **Google Cloud Platform**(GCP) and **Microsoft Azure** with **738** and **454** participants.


**Observation -**

Developers enjoys **AWS**, **GCP** and **Microsoft Azure** cloud service platforms more than other cloud platforms. These cloud providers are having amazing services which works from **IaaS** to **SaaS**. And thses platforms comes in handy in modern Industries.

## **Q29**

#### **Present - Q29_A - Do you use any of the following cloud computing products on a regular basis?**

#### **Future - Q29_B - In the next 2 years, do you hope to become more familiar with any of these specific cloud computing products?**

In [ ]:
data["Q29_A_Part_1"].unique(), data["Q29_B_Part_1"].unique()

In [ ]:
# Q29_A and Q29_B
cc_plt = clean(df, 'Q29_A_Part_', 5).rename(columns = {'Tools':'Cloud Platforms'}).sort_values(by= "Counts"
                                                                                                 ,ascending=False)
ccp = clean(df, "Q29_B_Part_", 5).rename(columns = {"Tools":"Cloud_Platforms"}).sort_values(by= "Counts"
                                                                                             ,ascending=False)

In [ ]:
# Sub-Bar Charts
title = 'Cloud Computing Platforms: Present-use vs Future-wish'

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing = 0.3,
                    shared_yaxes=False,
                   )

fig.add_trace(go.Bar(y=cc_plt['Cloud Platforms'],
                     x=cc_plt["Counts"],
                     orientation= 'h',
                     name="Present-Use",
                     marker_color='lightseagreen',
                     opacity =0.6
                    ),row=1, col=1,
             )

fig.add_trace(go.Bar(y = ccp['Cloud_Platforms'],
                     x = ccp['Counts'],
                     orientation= 'h',
                     name="In Next 2 years",
                     marker_color='gold',
                     opacity =0.6
                    ),row=1, col=2,
             )

fig.update_traces(marker_line_color='black',
              marker_line_width=1.5)

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.2,
                  height=450, width=950,
                  barmode='group',
                  titlefont={'size': 28},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')
                 

fig.update_layout(xaxis_title='Count',yaxis_linewidth=0,
autosize=False,
margin=dict(
    l=210, r=150, b=100, t=70,
),
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

**Facrts -**

- **In Present -**
  - **2575** participants are using **Other Cloud Platforms**
  - **2270** participants are using **Amazon Elastic Compute Cloud** (**EC2**)
  - **1960** participants are using **Google Cloud Compute Engine**
  - **1503** participants are using **Microsoft Azure Virtual Machines**
  

- **In next 2_Years -**
  - **7497** participants wants to learn **Google Cloud Compute Engine**
  - **5793** participants wants to learn **Microsoft Azure Virtual Machines**
  - **5373** participants wants to learn **Amazon Elastic Compute Cloud**
  - **2575** participants wants to learn **Other Cloud Platforms
  
  
  
**Observation -**

As we can see from above Bar charts, 
- **In Present**, **EC2 (Amazon)**, **Google Cloud Compute Engine**, **Microsoft Azure Virtual Machines** are used mostly and the reason for their use is either these platforms are **Easy and User Friendly** or **Widely Used in Industries**.


- **In Next 2 years**, same 3 cloud compute platforms are using but **EC2** of Amazon are given 3rd Priority to learn than other 2 cloud compute. And also we can say, **in next 2 years** **Google Cloud Compute Engine** will more preferred over **Amzon's EC2**, and followed by **Microsoft Azure Virtual Machine**

### **Total Cloud Computing Platforms Use per Participants**

In [ ]:
# Creating New Dataframe for Q29-A and Q29-B
q29A = df.iloc[:, 142:147].drop('Q29_A_Part_4', axis = 1)
q29B = df.iloc[:, 280:285].drop('Q29_B_Part_4', axis = 1)

# Applying RemoveSpaceBetStrings function to replace internal whitespaces between words
RemoveSpaceBetStrings(q29A)
RemoveSpaceBetStrings(q29B)

# Cloud Platform counts uses by participants
ccp_a = counts(q29A).rename(columns = {"Tools":"Cloud_Computing_Platform"})
ccp_b = counts(q29B).rename(columns = {"Tools":"Cloud_Computing_Platform"})

In [ ]:
val_29A = ccp_a['Counts'].value_counts()
val_29b = ccp_b['Counts'].value_counts()

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=val_29A.index
                     ,values=val_29A.values
                     ,name="Present Use")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=val_29b.index
                     ,values=val_29b.values
                     ,name="In Next 2 years")
              ,row=1
              ,col=2)

fig.update_layout(title="Total Cloud Computing Platforms per Participants : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
              ,template='simple_white'
              ,paper_bgcolor='#F5F5F5'
              ,plot_bgcolor='#F5F5F5'
            ).update_yaxes(categoryorder='total ascending')

**Facts -**

- In Present,
  - Any **1** Cloud Computing platorm are used by **3352** participants which are **75.5%**
  - Any **2** Cloud Computing platorm are used by **785** participants which are **17.7%**
  - Any **3**  Cloud Computing platorm are used by **299** participants which are **6.74%**
  

- In next 2Years, 
  - Any **1**  Cloud Computing platorm are used by **3367** participants which are **36%**
  - Any **3**  Cloud Computing platorm are used by **3359** participants which are **35.9%**
  - Any **2**  Cloud Computing platorm are used by **2621** participants which are **28%**
  
  
**Observation -**

**In present**, many participants are using any **1** to **3** cloud computing platforms either in their companies or for practicing purpose. **In next 2 years**, many participants are willing to learn any **1** to **3** cloud computing platforms either **to enter in Data Science field** or **Upskill existing skills**

## **Q30 -**


#### **Present** - Q30A **Do you use any of the following data storage products on a regular basis?**
#### **Future** - Q30B **In the next 2 years, do you hope to become more familiar with any of these specific data storage products?** 


**Note -** Q30-B consists as **`np.nan`**. As it does not has any value in it. So we can not compare it with present values.

In [ ]:
data['Q30_B_Part_1'].unique(), data['Q30_B_Part_1'].unique()

In [ ]:
data_stg_A = clean(df, "Q30_A_Part_", 8).rename(columns = {"Tools" : "Data_Storage_Product"}).sort_values(by = "Counts")
# data_stg_B = clean(df, "Q30_B_Part_", 8).rename(columns = {"Tools" : "Data_Storage_Product"}).sort_values(by = "Counts")

In [ ]:
fig = px.bar(data_stg_A, y= "Data_Storage_Product", x = "Counts")
fig.show()

**Facts -**

- **2575** participants using **Other** Data Storage products regularly.
- **2308** participants using **Amazon Simple Storage Service (S3)** Data Storage products regularly.
- **1950** participants using **Google Cloud Storage (GCS)** Data Storage products regularly.
- **908** participants using **Microsoft Azure DataLake Storage** Data Storage products regularly.
- **841** participants using **Microsoft Azure Disk Storage** Data Storage products regularly.
- **772** participants using **Google Cloud Filestore** Data Storage products regularly.
- **685** participants using **Amazon Elastic File System (EFS)** Data Storage products regularly.


**Observation -**

As we can see from above Bar chart, If we combine **Data Storage Products** provided by **Companies**. We can see **Amazon Data Storage** products are in high demands and used widely in industries, which consists **Amazon Simple Storage Service** (**S3**) and **Amazon Elastic File System** (**EFS**). Followed by **Google Data Storage** products, which consists **Google Cloud Stoarage** (**GCS**) and **Google Cloud Filestore**, and **Microsoft Azure Data Storage** products, which consists **Microsoft Azure Data Lake Storage** and **Microsoft Azure Disk Storage**.


These **3 Cloud Companies** cloud data storage are highly and widely used in many industries to store data and they provide separate data storage facility on the basis of **Pay as You Use**, it means we only have to pay as we use till date.

### Total Data Storage Counts per participants

In [ ]:
q30A = df.iloc[:, 147:155].drop('Q30_A_Part_7', axis = 1)

# Removing whitespaces
RemoveSpaceBetStrings(q30A)

d_store = counts(q30A).rename(columns = {"Tools":"Data_Storage_Counts"})

In [ ]:
d_store_cnt = d_store['Counts'].value_counts()

# Pie Chart
fig = px.pie(d_store_cnt
             ,names = d_store_cnt.index
             ,values = d_store_cnt.values
             ,title = "Data Storeage Counts used regularly by Participants"
             )

fig.show()

**Facts -**

- **2776** participants using any **1** Data Storage with **59.3%**
- **1327** participants using any **2** Data Storage with **28.4%**
- **346** participants using any **3** Data Storage with **7.39%**
- **129** participants using any **4** Data Storage with **2.76%**
- **67** participants using any **6** Data Storage with **1.43%**
- **34** participants using any **5** Data Storage with **0.727%**


**Observation -**

Data Storage users using any **1** or **2** data storage services providers. These data storage are either widely used or high in demand.

## **Q31 -**


#### **Present** - **Q31 A - Do you use any of the following managed machine learning products on a regular basis?**


#### **Future** - **Q31 B - In the next 2 years, do you hope to become more familiar with any of these managed machine learning products?**

In [ ]:
data['Q31_A_Part_1'].unique(), data['Q31_B_Part_1'].unique()

In [ ]:
ml_pro_A = clean(df, "Q31_A_Part_", 10).rename(columns = {"Tools": "Managed_ML_Products"})
ml_pro_B = clean(df, "Q31_B_Part_", 10).rename(columns = {"Tools": "Managed_ML_Products"})

In [ ]:
# Sub-Bar Charts
title = 'Managed Machine Learning Products : Present-use vs Future-wish'

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing = 0.3,
                    shared_yaxes=False,
                   )

fig.add_trace(go.Bar(y=ml_pro_A['Managed_ML_Products'],
                     x=ml_pro_A["Counts"],
                     orientation= 'h',
                     name="Present-Use",
                     marker_color='lightseagreen',
                     opacity =0.6
                    ),row=1, col=1,
             )

fig.add_trace(go.Bar(y = ml_pro_B['Managed_ML_Products'],
                     x = ml_pro_B['Counts'],
                     orientation= 'h',
                     name="In Next 2 years",
                     marker_color='gold',
                     opacity =0.6
                    ),row=1, col=2,
             )

fig.update_traces(marker_line_color='black',
              marker_line_width=1.5)

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.2,
                  height=450, width=950,
                  barmode='group',
                  titlefont={'size': 28},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')
                 

fig.update_layout(xaxis_title='Count',yaxis_linewidth=0,
autosize=False,
margin=dict(
    l=210, r=150, b=100, t=70,
),
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

**Facts -**

In present,
  - **6381** participats does not use any manage machine learning systems.
  
  
  - **Amazon SageMaker** are used regularly by **991** participants.
  - **Azure Machine Learning Studio** are used regularly by **945** participants.
  - **Databricks** are used regularly by **825** participants.
  - **Google Cloud Vertex AI** are used regularly by **714** participants.
  - **DataRobot** are used regularly by **332** participants.
  - **Rapidminer** are used regularly by **293** participants.
  
  
In Next 2years,
- **Google Cloud Vertex AI** are planned to use by **5304** participants.
- **Azure Machine Learning Studio** are planned to use by **5029** participants.
- **Amazon SageMaker** are planned to use by **3649** participants.
- **Databricks** are planned to use by **2202** participants.


**Observations -**

Currently, we can see in above bar chart, cloud based Machine Learning are less used by Machine Learners. But in next 2 years, more people will use it regularly as compared to present.

### **Total Managed Machine Learning Products Count used or desired to work by Participants**

In [ ]:
q31A = df.iloc[:, 155:165].drop("Q31_A_Part_9", axis = 1)
q31B = df.iloc[:, 293:303].drop("Q31_B_Part_9", axis = 1)


# Removing Whitespaces
RemoveSpaceBetStrings(q31A)
RemoveSpaceBetStrings(q31B)

# Applying counts function
ml_prod_cnt_A = counts(q31A).rename({"Tools" :"Managed_ML_Products"})
ml_prod_cnt_B = counts(q31B).rename({"Tools" :"Managed_ML_Products"})


# Taking value counts of "Counts" of both dataframes

mml_val_a = ml_prod_cnt_A['Counts'].value_counts()
mml_val_b = ml_prod_cnt_B['Counts'].value_counts()

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=mml_val_a.index
                     ,values=mml_val_a.values
                     ,name="Present Use")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=mml_val_b.index
                     ,values=mml_val_b.values
                     ,name="In Next 2 years")
              ,row=1
              ,col=2)

fig.update_layout(title="Total Managed ML Products : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
              ,template='simple_white'
              ,paper_bgcolor='#F5F5F5'
              ,plot_bgcolor='#F5F5F5'
            ).update_yaxes(categoryorder='total ascending')

**Observations -**

- In present, any **1** managed ML products are used by **68.5%** users. Followed by any **2** or **3** ML products.
  - Here, we can say any **1** or **2** or **3** managed ML products are used most users to work on Machine Learning in cloud.

## **Q32 -**


#### **Present** - **Q32 A - Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis?**



#### **Future** - **Q32 B - Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you hope to become more familiar with in the next 2 years?**

In [ ]:
data['Q32_A_Part_1'].unique(), data['Q32_B_Part_1'].unique()

In [ ]:
big_data_A = clean(df, "Q32_A_Part_", 21).rename(columns = {"Tools" : "Big_Data_Products"})
big_data_B = clean(df, "Q32_B_Part_", 21).rename(columns = {"Tools" : "Big_Data_Products"})

In [ ]:
# Sub-Bar Charts
title = 'BigData Products : Present-use vs Future-wish'

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing = 0.3,
                    shared_yaxes=False,
                   )

fig.add_trace(go.Bar(y=big_data_A['Big_Data_Products'],
                     x=big_data_A["Counts"],
                     orientation= 'h',
                     name="Present-Use",
                     marker_color='lightseagreen',
                     opacity =0.6
                    ),row=1, col=1,
             )

fig.add_trace(go.Bar(y = big_data_B['Big_Data_Products'],
                     x = big_data_B['Counts'],
                     orientation= 'h',
                     name="In Next 2 years",
                     marker_color='gold',
                     opacity =0.6
                    ),row=1, col=2,
             )

fig.update_traces(marker_line_color='black',
              marker_line_width=1.5)

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.2,
                  height=600, width=950,
                  barmode='group',
                  titlefont={'size': 28},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')
                 

fig.update_layout(xaxis_title='Count',yaxis_linewidth=0,
autosize=False,
margin=dict(
    l=210, r=150, b=100, t=70,
),
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

**Observations -**


- **In Present,**
  - **MySQL** are most favorite **BigData** products.
  - followed by **PostgreSQL**, **Microsoft SQL Server**, **MongoDB**, these BigData products are most widely used by participants in present, and these BigData products are even widely used in Industries also.

- **In Next 2 years,**
   - **MySQL** are most favorite to leaners and at top priority.
   - followed by **MongoDB**, **PostgreSQL**, **Google Cloud SQL** and **Microsoft SQL Server**
   
   
   
We can say that, in most industries **MySQL** are used more than other **SQL**s and **MongoDB** are used more than other **NoSQL**s. And these skill sets are having ampale of demands than other SQL or NoSQL BigData products.

### **Total BigData Products Counts Per Participants**

In [ ]:
q32A = df.iloc[:, 165:186].drop("Q32_A_Part_20", axis = 1)
q32B = df.iloc[:, 303:324].drop("Q32_B_Part_20", axis = 1)


# Removes Whitespaces
RemoveSpaceBetStrings(q32A)
RemoveSpaceBetStrings(q32B)

# Applying "counts function"
bd_cnt_a = counts(q32A).rename({"Tools" :"BigData Products"})
bd_cnt_b = counts(q32B).rename({"Tools" :"BigData Products"})

# Replacing 14, 15, 17 and 18 with "14" 
# bd_cnt_a['Counts'] = bd_cnt_a['Counts'].replace([14, 15, 17,18], 14)
# bd_cnt_a['Counts'] = bd_cnt_a['Counts'].replace([14, 15, 17,18], 14)

# taking value counts for "Counts" attribute from both dataframes
bd_val_a = bd_cnt_a['Counts'].value_counts().sort_values()
bd_val_b = bd_cnt_b['Counts'].value_counts().sort_values()

In [ ]:
# Sub-Bar Charts
title = 'Total BigData Products Counts per Participants : Present-use vs Future-wish'

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing = 0.3,
                    shared_yaxes=False,
                   )

fig.add_trace(go.Bar(y=bd_val_a.index,
                     x=bd_val_a.values,
                     orientation= 'h',
                     name="Present-Use",
                     marker_color='lightseagreen',
                     opacity =0.6
                    ),row=1, col=1,
             )

fig.add_trace(go.Bar(y = bd_val_b.index,
                     x = bd_val_b.values,
                     orientation= 'h',
                     name="In Next 2 years",
                     marker_color='gold',
                     opacity =0.6
                    ),row=1, col=2,
             )

fig.update_traces(marker_line_color='black',
              marker_line_width=1.5)

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.2,
                  height=700, width=950,
                  barmode='group',
                  titlefont={'size': 28},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')
                 

fig.update_layout(xaxis_title='Count',yaxis_linewidth=0,
autosize=False,
margin=dict(
    l=210, r=150, b=100, t=70,
),
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

**Observation -**

- In present, from any 1 BigData to any 3 BigData products are used by users simultaneously.
- In next 2 years, from any 1 to any 4 Bigdata products are learning plans of users.

## **Q33** 

- **Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often?**

In [ ]:
data['Q33'].unique()

In [ ]:
val = df['Q33'].value_counts(dropna=True).sort_values()

fig = px.bar(val
             ,x = val.values
             ,y = val.index
             ,title = "Big Data Prodcuts use most often"
             ,labels={
                 "x" : "frequency"
                 ,"index": "Big Data Products"
             }
            )

fig.show()

**Observations -**

- **MySQL** is **SQL** BigData product and it is high in demands from most industries.
- followed by **PostgreSQL** and **Microsoft SQL Server**
- **MongoDB** is **NoSQL** BigData product and also used widely in most industries for **NoSQL**.

## **Q34**

- **Present** - **Which of the following business intelligence tools do you use on a regular basis?**
- **Future** - **Which of the following business intelligence tools do you hope to become more familiar with in the next 2 years?**

In [ ]:
data['Q34_A_Part_1'].unique(), data['Q34_B_Part_1'].unique()

In [ ]:
bi_A = clean(df, "Q34_A_Part_", 17).rename(columns = {"Tools":"BI_Tools"})
bi_B = clean(df, "Q34_A_Part_", 17).rename(columns = {"Tools":"BI_Tools"})

In [ ]:
# Sub-Bar Charts
title = 'Business Intelligence Tools : Present-use vs Future-wish'

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing = 0.3,
                    shared_yaxes=False,
                   )

fig.add_trace(go.Bar(y=bi_A['BI_Tools'],
                     x=bi_B['Counts'],
                     orientation= 'h',
                     name="Present-Use",
                     marker_color='lightseagreen',
                     opacity =0.6
                    ),row=1, col=1,
             )

fig.add_trace(go.Bar(y = bi_B["BI_Tools"],
                     x = bi_B["Counts"],
                     orientation= 'h',
                     name="In Next 2 years",
                     marker_color='gold',
                     opacity =0.6
                    ),row=1, col=2,
             )

fig.update_traces(marker_line_color='black',
              marker_line_width=1.5)

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.2,
                  height=500, width=950,
                  barmode='group',
                  titlefont={'size': 28},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')
                 

fig.update_layout(xaxis_title='Count',yaxis_linewidth=0,
autosize=False,
margin=dict(
    l=210, r=150, b=100, t=70,
),
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

### **Total Business Intelligence Counts per Participants**

In [ ]:
q34A = df.iloc[:, 187:204].drop('Q34_A_Part_16', axis = 1)
q34B = df.iloc[:, 324:341].drop('Q34_B_Part_16', axis = 1)

# Removing Whitespaces
RemoveSpaceBetStrings(q34A)
RemoveSpaceBetStrings(q34B)

# Applying counts functions
bi_cnt_a = counts(q34A).rename(columns = {"Tools" : "Total_BI_Tools"})
bi_cnt_b = counts(q34B).rename(columns = {"Tools" : "Total_BI_Tools"})

# Taking value counts
bi_val_a = bi_cnt_a['Counts'].value_counts()
bi_val_b = bi_cnt_b['Counts'].value_counts()

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=bi_val_a.index
                     ,values=bi_val_a.values
                     ,name="Present Use")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=bi_val_b.index
                     ,values=bi_val_b.values
                     ,name="In Next 2 years")
              ,row=1
              ,col=2)

fig.update_layout(title="Total BI Tools use per Participants : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
              ,template='simple_white'
              ,paper_bgcolor='#F5F5F5'
              ,plot_bgcolor='#F5F5F5'
            ).update_yaxes(categoryorder='total ascending')

## **Q35 - Which of the following business intelligence tools do you use most often?**

In [ ]:
data['Q35'].unique()

In [ ]:
bi_tool_val = df['Q35'].value_counts().sort_values()

fig = px.bar(bi_tool_val
             ,x = bi_tool_val.values
             ,y = bi_tool_val.index
             ,labels={
                 "x" : "frequencies"
                 ,"y" : "BI_Tools"
             }
             ,title = "Business Intelligence Tools use most often"
            )

fig.show()

## **Q36**


- **Present -** **Q36 A** Do you use any automated machine learning tools (or partial AutoML tools) on a regular basis?
- **Future -** **Q36 B** Which categories of automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years?

In [ ]:
data["Q36_A_Part_7"].unique(), data["Q36_B_Part_7"].unique()

In [ ]:
df['Q36_A_Part_7'] = df['Q36_A_Part_7'].replace('No / None', 'None')

In [ ]:
auto_ml_36a = clean(df, "Q36_A_Part_", 8).rename(columns = {"Tools":"Auto ML Tools"})#.sort_values(by = "Counts", ascending = True)
auto_ml_36b = clean(df, "Q36_B_Part_", 8).rename(columns = {"Tools":"Auto ML Tools"})

In [ ]:
# Sub-Plots
title = "Automated Machine Learning Tools: Present-use vs Future-plan"

fig = make_subplots(rows=1
                    ,cols=2
                    ,horizontal_spacing=0.3
                    ,shared_yaxes=True)

fig.add_trace(go.Bar(y = auto_ml_36a['Auto ML Tools']
                     ,x = auto_ml_36a['Counts']
                     ,orientation='h'
                     ,name="Present"
                     ,marker_color = "lightseagreen"
                     ,opacity=0.5
                    )
              ,row=1
              ,col=1
             )

fig.add_trace(go.Bar(y = auto_ml_36b['Auto ML Tools']
                     ,x = auto_ml_36b['Counts']
                     ,orientation ='h'
                     ,name = "Next_2Yr"
                     ,marker_color = "gold"
                     ,opacity=0.5
                    )
              ,row=1
              ,col=2
             )

fig.update_traces(marker_line_color = "black"
                  ,marker_line_width = 1.5
                 )

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.15,
                  height=450, width=950,
                  barmode='group',
                  titlefont={'size': 24},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=0.75, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              ).update_yaxes(categoryorder='total ascending')

fig.update_layout(xaxis_title = "Counts"
                  ,yaxis_linewidth = 0
                  ,autosize = False
                  ,margin = dict(l = 100
                                 ,r = 75
                                 ,b = 70
                                 ,t = 70
                                )
                 )

fig.update_xaxes(showgrid = False)
fig.update_yaxes(showgrid = False)
fig.show()

**Facts -**

### **Total Auto-Machine Learning Tools used per participants**

In [ ]:
q36A = df.iloc[:, 205:213].drop("Q36_A_Part_7", axis = 1)
q36B = df.iloc[:, 341:349].drop("Q36_B_Part_7", axis = 1)

# Removing Whitespaces
RemoveSpaceBetStrings(q36A)
RemoveSpaceBetStrings(q36B)

# Applying counts functions
automl_a_cnt = counts(q36A).rename(columns = {"Tools" : "Auto_ML_Tools"})
automl_b_cnt = counts(q36B).rename(columns = {"Tools" : "Auto_ML_Tools"})

# Taking value counts
automl_val_a = automl_a_cnt['Counts'].value_counts()
automl_val_b = automl_b_cnt['Counts'].value_counts()

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=automl_val_a.index
                     ,values=automl_val_a.values
                     ,name="Present")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=automl_val_b.index
                     ,values=automl_val_b.values
                     ,name="Next_2years")
              ,row=1
              ,col=2)

fig.update_layout(title="Total Auto-ML Tools per Participants : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
            ).update_yaxes(categoryorder='total ascending')

**Facts -**

## **Q37**

- **Present -** **Q37 A** Which of the following automated machine learning tools (or partial AutoML tools) do you use on a regular basis?
- **Future -** **Q37 B** Which specific automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years?

In [ ]:
df['Q37_A_Part_7'].unique(), df['Q37_B_Part_7'].unique()

In [ ]:
# Replacing 'No / None' with 'None'
df['Q37_A_Part_7'] = df['Q37_A_Part_7'].replace('No / None', 'None')

# Applying "clean function" on respective columns
spc_automl_37A = clean(df, "Q37_A_Part_"
                       ,8).rename(columns = {"Tools":"Specific_AutoML_Tools"}).sort_values(by = 'Specific_AutoML_Tools')
spc_automl_37B = clean(df, "Q37_B_Part_"
                       ,8).rename(columns = {"Tools":"Specific_AutoML_Tools"}).sort_values(by = 'Specific_AutoML_Tools')

In [ ]:
# Sub-Plots
title = "Specific Automated-ML Tools: Present-use vs Future-plan"

fig = make_subplots(rows=1
                    ,cols=2
                    ,horizontal_spacing=0.1
                    ,shared_yaxes=False)

fig.add_trace(go.Bar(y = spc_automl_37A['Specific_AutoML_Tools']
                     ,x = spc_automl_37A['Counts']
                     ,orientation='h'
                     ,name="Present"
                     ,marker_color = "lightseagreen"
                     ,opacity=0.5
                    )
              ,row=1
              ,col=1
             )

fig.add_trace(go.Bar(y = spc_automl_37B['Specific_AutoML_Tools']
                     ,x = spc_automl_37B['Counts']
                     ,orientation ='h'
                     ,name = "Next_2Yr"
                     ,marker_color = "gold"
                     ,opacity=0.5
                    )
              ,row=1
              ,col=2
             )

fig.update_traces(marker_line_color = "black"
                  ,marker_line_width = 1.5
                 )

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.15,
                  height=450, width=950,
                  barmode='group',
                  titlefont={'size': 24},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=0.75, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              )

fig.update_layout(margin = dict(l = 150
                                 ,r = 150
                                 ,b = 70
                                 ,t = 100
                                )
                 )

fig.show()

**Facts -**

### **Total Specific Auto-ML Use per participants**

In [ ]:
q37A = df.iloc[:, 213:221].drop("Q37_A_Part_7", axis = 1)
q37B = df.iloc[:, 349:357].drop("Q37_B_Part_7", axis = 1)

# Removing Whitespaces
RemoveSpaceBetStrings(q37A)
RemoveSpaceBetStrings(q37B)

# Applying counts functions
spc_automl_a = counts(q37A).rename(columns = {"Tools" : "Auto_ML_Tools"})
spc_automl_b = counts(q37B).rename(columns = {"Tools" : "Auto_ML_Tools"})

# Taking value counts
spc_a_val = spc_automl_a['Counts'].value_counts()
spc_b_val = spc_automl_b['Counts'].value_counts()

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=spc_a_val.index
                     ,values=spc_a_val.values
                     ,name="Present")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=spc_b_val.index
                     ,values=spc_b_val.values
                     ,name="Next_2years")
              ,row=1
              ,col=2)

fig.update_layout(title="Total Specific Auto-ML Tools : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
            ).update_yaxes(categoryorder='total ascending')

**Facts -**

## **Q38**

- **Present -** **Q38 A** Do you use any tools to help manage machine learning experiments?
- **Future -** **Q38 B** In the next 2 years, do you hope to become more familiar with any of these tools for managing ML experiments?

In [ ]:
data['Q38_A_Part_11'].unique(), data['Q38_B_Part_11'].unique()

In [ ]:
# Replacing "No / None" with "None"
df["Q38_A_Part_11"] = df["Q38_A_Part_11"].replace('No / None', "None")

# Applying "clean Function" on respective questions
manage_ml_A = clean(df, "Q38_A_Part_"
                    ,12).rename(columns = {"Tools":"ML Managing Tools"}).sort_values(by = "Counts", ascending = False)

manage_ml_B = clean(df, "Q38_B_Part_"
                    ,12).rename(columns = {"Tools":"ML Managing Tools"}).sort_values(by = "Counts", ascending = False)

In [ ]:
# Sub-Plots
title = "Tools to Manage ML Experiments: Present-use vs Future-plan"

fig = make_subplots(rows=1
                    ,cols=2
                    ,horizontal_spacing=0.1
                    ,shared_yaxes=False)

fig.add_trace(go.Bar(y = manage_ml_A['ML Managing Tools']
                     ,x = manage_ml_A['Counts']
                     ,orientation='h'
                     ,name="Present"
                     ,marker_color = "lightseagreen"
                     ,opacity=0.5
                    )
              ,row=1
              ,col=1
             )

fig.add_trace(go.Bar(y = manage_ml_B['ML Managing Tools']
                     ,x = manage_ml_B['Counts']
                     ,orientation ='h'
                     ,name = "Next_2Yr"
                     ,marker_color = "gold"
                     ,opacity=0.5
                    )
              ,row=1
              ,col=2
             )

fig.update_traces(marker_line_color = "black"
                  ,marker_line_width = 1.5
                 )

fig.update_layout(title=title, 
                  font_family="San Serif",
                  bargap=0.15,
                  height=450, width=950,
                  barmode='group',
                  titlefont={'size': 24},
                  template='simple_white',
                  paper_bgcolor='#F5F5F5',
                  plot_bgcolor='#F5F5F5',
                  legend=dict(orientation="v", 
                              y=1, 
                              yanchor="top", 
                              x=1.250, 
                              xanchor="right",)                 
              )

fig.update_layout(margin = dict(l = 150
                                 ,r = 150
                                 ,b = 70
                                 ,t = 100
                                )
                 )

fig.show()

**Facts -**

### **Total use**

In [ ]:
q38A = df.iloc[:, 221:233].drop("Q38_A_Part_11", axis = 1)
q38B = df.iloc[:, 357:].drop("Q38_B_Part_11", axis = 1)

# Removing whitespaces
RemoveSpaceBetStrings(q38A)
RemoveSpaceBetStrings(q38B)

# Applying counts function
ml_exp_tools_a = counts(q38A).rename(columns = {"Tools":"ML Experiment Managing Tools"})
ml_exp_tools_b = counts(q38B).rename(columns = {"Tools":"ML Experiment Managing Tools"})

In [ ]:
ml_exp_cnt_a = ml_exp_tools_a['Counts'].value_counts()
ml_exp_cnt_b = ml_exp_tools_b['Counts'].value_counts()


# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1
                    ,cols=2
                    ,specs=[[{'type':'domain'}
                             ,{'type':'domain'}]]
                    ,subplot_titles=['Present'
                                     ,'Future']
                   )

fig.add_trace(go.Pie(labels=ml_exp_cnt_a.index
                     ,values=ml_exp_cnt_a.values
                     ,name="Present")
              ,row=1
              ,col=1)

fig.add_trace(go.Pie(labels=ml_exp_cnt_b.index
                     ,values=ml_exp_cnt_b.values
                     ,name="Next_2years")
              ,row=1
              ,col=2)

fig.update_layout(title="Total ML Experiment Tools : Present vs. Future"
              ,font_family="San Serif"
              ,height=500, width=975
              ,titlefont={'size': 28}
            ).update_yaxes(categoryorder='total ascending')

**Facts -**

## **Q39**


**Where do you publicly share your data analysis or machine learning applications?**

In [ ]:
data['Q39_Part_1'].unique()

In [ ]:
data_anal_platform = clean(df, "Q39_Part_"
                           ,10).rename(columns = {"Tools":"DA/ML Application Sharing Platforms"}).sort_values(by = "Counts")
data_anal_platform

In [ ]:
fig = px.bar(data_anal_platform
             ,x = "Counts"
             ,y = "DA/ML Application Sharing Platforms"
             ,title = "Data Analysis / ML App. Sharing Platforms"
             ,
            )
fig.show()

### **Total DA or ML Application Sharing Platforms**

In [ ]:
q39 = df.iloc[:, 233:243]

# Removing Whiteshpaces from columns
RemoveSpaceBetStrings(q39)

# Applying "CountsFunction"
trans_39 = counts(q39).rename(columns = {"Tools": 'Total DA/ML App Sharing Platforms'})

app_sh_cnt = trans_39['Counts'].value_counts()

In [ ]:
# Pie Chart

fig = px.pie(app_sh_cnt
             ,names=app_sh_cnt.index
             ,values=app_sh_cnt.values
             ,title="Total DA / ML Application Sharing Platforms used by participants"
            )

fig.show()

**Facts -**

## **Q40**


**On which platforms have you begun or completed data science courses?**

In [ ]:
data['Q40_Part_1'].unique()

In [ ]:
online_course = clean(df, "Q40_Part_"
                      ,12).rename(columns = {"Tools":"Online_Course_Platforms"}).sort_values(by = 'Counts')


# Bar Plot 
fig = px.bar(online_course
             ,x = "Counts"
             ,y = "Online_Course_Platforms"
             ,title = "Online Courses Platorms used to Learn Data Science"
            )

fig.show()

**Facts -**

### **Total Online Course Platforms used to Learn Data Science**

In [ ]:
q40 = df.iloc[:, 243:255].drop('Q40_Part_11', axis = 1)

# Applying RemoveSpaceBetStrings function on dataframe
RemoveSpaceBetStrings(q40)

# Applying "counts function" on dataframe
onl_course = counts(q40).rename(columns = {"Tools":"Total Online Platforms used to Learn DS"})

onl_curs_cnt = onl_course['Counts'].value_counts()


# Pie Chart
fig = px.pie(onl_curs_cnt
             ,names=onl_curs_cnt.index
             ,values=onl_curs_cnt.values
             ,title = "Total Online Courses Platforms used by Learners"
            )

fig.show()

**Facts -**

# **Q41**


**What is the primary tool that you use at work or school to analyze data?**

In [ ]:
data['Q41'].unique()

In [ ]:
q41_val = df['Q41'].value_counts().sort_values()

# Bar Chart
fig = px.bar(q41_val
             ,x = q41_val.values
             ,y = q41_val.index
             ,title="Primary Tools used to Analyze Data at Work or School"
             ,labels={"x" : "frequencies"
                      ,"index" : "Primary Tools used for Analyze Data"
                     }
            )

fig.show()

# **Q42**

**Who/what are your favorite media sources that report on data science topics?**

In [ ]:
data["Q42_Part_11"].unique()

In [ ]:
ds_report_src = clean(df, "Q42_Part_"
                      ,11).rename(columns={"Tools":"DS Report Source"}).sort_values(by = 'Counts')

# Bar Chart
fig = px.bar(ds_report_src
             ,x = "Counts"
             ,y = "DS Report Source"
             ,title = "Data Science Report Source"
            )

fig.show()

**Facts -**

### Total DS Source Reports used by Data Scientist

In [ ]:
q42 = df.iloc[:, 256:268].drop("Q42_Part_11", axis = 1)

# Applying RemoveSpaceBetStrings function dataframe
RemoveSpaceBetStrings(q42)

# Applying "counts" function on dataframe
src_report_ds = counts(q42).rename(columns = {"Tools":"Total DS Source Reports"})

src_rt_val = src_report_ds['Counts'].value_counts()


# Pie Chart
fig = px.pie(src_rt_val
             ,names = src_rt_val.index
             ,values = src_rt_val.values
             ,title = "Total DS Source Reports used by Data Scientist"
            )
fig.show()

**Facts -**